# EMA Registries: Nearest Neighbor Assignment (Option 1)

This notebook implements and analyzes the nearest neighbor assignment approach for landing EMA registries into existing clusters.

## 1. Setup & Imports

In [1]:
import os
import os
working_dir = "/home/gpinon/more_europa/clean_rdc_experiments/projects/P05_refine_dedup"
os.chdir(working_dir)
print(f"Changed working directory to {working_dir}")
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

from src.p05_refine_dedup import config
from src.p05_refine_dedup.utils.s3_io_functions import load_parquet_from_s3

Changed working directory to /home/gpinon/more_europa/clean_rdc_experiments/projects/P05_refine_dedup


## 2. Load Data

In [2]:
# a. Load registry embeddings from S3
s3_input_embeddings = "registry_data_catalog_experiments/P05_refine_dedup/registry_names_embeddings.parquet"
bucket_name = config.BUCKET_NAME_DEV
folder_path = s3_input_embeddings.rsplit('/', 1)[0]
file_name = s3_input_embeddings.rsplit('/', 1)[-1]
embeddings_df = load_parquet_from_s3(
    bucket_name=bucket_name,
    folder_path=folder_path,
    file_name=file_name,
)
print(f'Loaded registry embeddings: {embeddings_df.shape}')

Loaded registry embeddings: (54335, 4)


In [3]:
# b. Load EMA registry embeddings from S3
s3_ema_embeddings = 'registry_data_catalog_experiments/P05_refine_dedup/ema_registry_names_embeddings.parquet'
ema_folder_path = s3_ema_embeddings.rsplit('/', 1)[0]
ema_file_name = s3_ema_embeddings.rsplit('/', 1)[-1]
ema_embeddings_df = load_parquet_from_s3(
    bucket_name=bucket_name,
    folder_path=ema_folder_path,
    file_name=ema_file_name,
)
print(f'Loaded EMA registry embeddings: {ema_embeddings_df.shape}')

Loaded EMA registry embeddings: (237, 4)


In [4]:
# c.define elements specific to the experiment
exp_name='new_eucl_0.2'
ema_prediction_results_xlsx=f"data/W02/R03_evaluate_model_performance_on_ema_registries/{exp_name}/ema_prediction_results.xlsx"
# make sure the directory exists
results_dir = Path(ema_prediction_results_xlsx).parent
results_dir.mkdir(parents=True, exist_ok=True)

In [5]:
dist_type = 'euclidean'# or 'cosine'
max_dist = 0.2 # maximum distance threshold for considering a match

In [6]:
# d. Load cluster assignments from local Excel
clusters_table_xlsx="data/W03/from_notebooks/R08_loop_grid_searchs_start_0/v1/clusters_table.xlsx"
clusters_df = pd.read_excel(clusters_table_xlsx)
print(f'Loaded clusters table: {clusters_df.shape}')
col_cluster_final = 'cluster_final'

Loaded clusters table: (54347, 36)


In [7]:
# additional preprocessing, dependent on the experiment and the selected clusters table to load
# Vectorize the assignment for popular noise items (solo clusters)
n_solo = 1
col_cluster_id = 'cluster_id' # 'cluster_final'
clusters_df[col_cluster_id] = clusters_df[col_cluster_final]
mask = clusters_df[col_cluster_final].isna() & (clusters_df['number_of_occurrences'] >= n_solo)
clusters_df.loc[mask, col_cluster_id] = [
    f"solo_{i+1}" for i in range(mask.sum())
]

## 3. Preprocessing
No normalization required. Ensure embeddings are in correct format.

In [8]:
# Ensure embeddings are numpy arrays
embeddings_df['full_name_embedding'] = embeddings_df['full_name_embedding'].apply(np.array)
ema_embeddings_df['full_name_embedding'] = ema_embeddings_df['full_name_embedding'].apply(np.array)
# clusters_df[col_cluster_id] = clusters_df[col_cluster_id].astype(str)
print('Embeddings and clusters preprocessed.')

Embeddings and clusters preprocessed.


## 4. Nearest Neighbor Search and Results Table

In [ ]:
# Prepare lookup tables
emb_matrix = np.vstack(embeddings_df['full_name_embedding'].values)
ema_matrix = np.vstack(ema_embeddings_df['full_name_embedding'].values)
# Add cluster info to embeddings_df
embeddings_with_clusters = embeddings_df.merge(clusters_df, on=['full_name', 'number_of_occurrences'], how='left')
# For each EMA registry, find nearest neighbor and cluster info
results = []
for idx, ema_row in tqdm(ema_embeddings_df.iterrows(), total=ema_embeddings_df.shape[0]):
    print('---')
    print(f"Registry: {ema_row['full_name']}")
    ema_emb = ema_row['full_name_embedding'].reshape(1, -1)
    if dist_type == 'cosine':
        dists = cosine_distances(ema_emb, emb_matrix)[0]
    elif dist_type == 'euclidean':
        dists = euclidean_distances(ema_emb, emb_matrix)[0]
    # Find the closest match
    min_idx = np.argmin(dists)
    closest_row = embeddings_with_clusters.iloc[min_idx]
    if dists[min_idx] > max_dist:
        # If the closest distance is greater than the threshold, assign None
        assigned_cluster = None
        print(f"EMA {ema_row['full_name']} has no close match within threshold {max_dist}")
        print(f"Closest match: {closest_row['full_name']} with distance {round(dists[min_idx], 3)}")
    else:
        assigned_cluster = closest_row[col_cluster_id]
        print(f"EMA {ema_row['full_name']} assigned to cluster {assigned_cluster} with distance {round(dists[min_idx], 3)}")
        # Get all aliases in the same cluster
        cluster_aliases = embeddings_with_clusters[embeddings_with_clusters[col_cluster_id] == assigned_cluster]
        total_aliases = cluster_aliases.shape[0]
        total_occurrences = cluster_aliases['number_of_occurrences'].sum()
        # Most popular alias
        n1_row = cluster_aliases.sort_values('number_of_occurrences', ascending=False).iloc[0]
    results.append({
        'ema_object_id': ema_row['object_id'],
        'ema_full_name': ema_row['full_name'],
        'assigned_cluster': assigned_cluster,
        'distance_to_closest': dists[min_idx],
        'closest': closest_row['full_name'],
        'closest_nb_occ': closest_row['number_of_occurrences'],
        # total aliases should be the number of unique aliases in the cluster, or 0
        'total_aliases': total_aliases if assigned_cluster else 0,
        'total_occurrences': total_occurrences if assigned_cluster else 0,
        'N1_alias': n1_row['full_name'] if assigned_cluster else None,
        'N1_alias_nb_occ': n1_row['number_of_occurrences'] if assigned_cluster else None,
    })
results_df = pd.DataFrame(results)
display(results_df.head())

  0%|          | 1/237 [00:00<00:30,  7.78it/s]

---
Registry: Clinical Practice Research Datalink (CPRD) GOLD (CPRD GOLD)
EMA Clinical Practice Research Datalink (CPRD) GOLD (CPRD GOLD) assigned to cluster 70_3 with distance 0.11800000071525574
---
Registry: Cancer Registry of Instituto Português de Oncologia do Porto Francisco Gentil, E.P.E (IPO-Porto Cancer Registry)


  1%|▏         | 3/237 [00:00<00:30,  7.55it/s]

EMA Cancer Registry of Instituto Português de Oncologia do Porto Francisco Gentil, E.P.E (IPO-Porto Cancer Registry) has no close match within threshold 0.2
Closest match: The Nationwide Danish 1905 Cohort Study (1905 Cohort Study) with distance 0.2939999997615814
---
Registry: European Registry of Patients with McArdle disease or other rare form of muscle Glycogenoses (EUROMAC)
EMA European Registry of Patients with McArdle disease or other rare form of muscle Glycogenoses (EUROMAC) assigned to cluster 3354 with distance 0.1459999978542328
---
Registry: The UK-Irish Atopic Eczema Systemic Therapy Register (A-STAR)


  2%|▏         | 5/237 [00:00<00:32,  7.20it/s]

...

100%|█████████▉| 236/237 [00:34<00:00,  7.25it/s]

EMA IADB.nl (IADB) has no close match within threshold 0.2
Closest match: Haute-Garonne Cancer Registry (FR-HGA) with distance 0.5799999833106995
---
Registry: French National Registry of Rare Diseases (Banque Nationale de Données Maladies Rares) (BNDMR)
EMA French National Registry of Rare Diseases (Banque Nationale de Données Maladies Rares) (BNDMR) has no close match within threshold 0.2
Closest match: French National Rare Disease Registry (BNDMR) with distance 0.21699999272823334
---
Registry: HTI – Hospital Treatment Insights (HTI)


100%|██████████| 237/237 [00:34<00:00,  6.84it/s]

EMA HTI – Hospital Treatment Insights (HTI) has no close match within threshold 0.2
Closest match: French TTP Registry (TTP) with distance 0.6840000152587891


,ema_object_id,ema_full_name,assigned_cluster,distance_to_closest,closest,closest_nb_occ,total_aliases,total_occurrences,N1_alias,N1_alias_nb_occ
0,028ac269-7f7f-56d3-8dd3-0bd93913d864,Clinical Practice Research Datalink (CPRD) GOL...,70_3,0.118468,Clinical Practice Research Datalink GOLD (CPRD...,11,2,12,Clinical Practice Research Datalink GOLD (CPRD...,11.0
1,0446b0c0-0bec-5999-8c59-f0e4107dabc3,Cancer Registry of Instituto Português de Onco...,None,0.294426,The Nationwide Danish 1905 Cohort Study (1905 ...,1,0,0,None,NaN
2,060d06a8-0a14-5b8b-9fe4-882f08412d7f,European Registry of Patients with McArdle dis...,3354,0.145912,European registry for patients with McArdle di...,2,2,3,European registry for patients with McArdle di...,2.0
3,060ede1e-35d6-5dab-9e1e-d7e1e2f78c2a,The UK-Irish Atopic Eczema Systemic Therapy Re...,0_1_24,0.105932,International Network of Obstetric Survey Syst...,1,3,4,Italian Obstetric Surveillance System (ItOSS),2.0
4,08920b82-907a-5d03-828e-f76f7b3c735e,syndena GmbH (former OncoTyrol) (syndena GmbH),None,0.598728,Cancer Registry of Tyrol (CRT),5,0,0,None,NaN


## 5. Save Results
Save the results DataFrame to Excel for further analysis.

In [10]:
results_df.to_excel(ema_prediction_results_xlsx, index=False)
print(f'Results saved to {ema_prediction_results_xlsx}')

Results saved to data/W02/R03_evaluate_model_performance_on_ema_registries/new_eucl_0.2/ema_prediction_results.xlsx


## 6. Analysis & Metrics

In [12]:
# reload results
exp_name='new_eucl_0.2'
ema_prediction_results_xlsx=f"data/W02/R03_evaluate_model_performance_on_ema_registries/{exp_name}/ema_prediction_results.xlsx"
results_df = pd.read_excel(ema_prediction_results_xlsx)

In [15]:
# Identify noise clusters: '0' or ending with '_0'
def is_noise(cluster):
    return str(cluster) == '0' or str(cluster).endswith('_0')

# Compute transformation rate: how many EMA registries were transformed into clusters (assigned_cluster Not None)
transformed_count = results_df['assigned_cluster'].notna().sum()
print(f"Transformation rate: {transformed_count}/ {results_df.shape[0]} ({transformed_count/results_df.shape[0]*100:.2f}%)")

# Number of distinct clusters with at least one EMA registry
distinct_clusters = results_df['assigned_cluster'].dropna().unique()
print(f"Distinct clusters with at least one EMA registry: {len(distinct_clusters)}")

# Clusters with multiple EMA registries
multiple_ema_clusters = results_df.groupby('assigned_cluster').filter(lambda x: len(x) > 1)
print(f"Clusters with multiple EMA registries: {multiple_ema_clusters['assigned_cluster'].nunique()}")
display(multiple_ema_clusters[['assigned_cluster']])

# Number of Aliases per transformed EMA registry (set to int)
results_df['total_aliases'] = results_df['total_aliases'].fillna(0).astype(int)
# Number of occurrences per transformed EMA registry (set to int)
results_df['total_occurrences'] = results_df['total_occurrences'].fillna(0).astype(int)
print(f"Mean Number of aliases per transformed EMA registry: {results_df['total_aliases'].mean():.2f}")
print(f"Mean Number of occurrences per transformed EMA registry: {results_df['total_occurrences'].mean():.2f}")

Transformation rate: 51/ 237 (21.52%)
Distinct clusters with at least one EMA registry: 51
Clusters with multiple EMA registries: 0


,assigned_cluster


Mean Number of aliases per transformed EMA registry: 0.95
Mean Number of occurrences per transformed EMA registry: 14.78


In [14]:
# display the 10 first rows of transformed clusters ranked by total_aliases
top_clusters = results_df[results_df['assigned_cluster'].notna()].sort_values('total_aliases', ascending=False).head(10)
print("Top 10 clusters by number of aliases:")
display(top_clusters[['assigned_cluster', 'total_aliases', 'total_occurrences']])

Top 10 clusters by number of aliases:


,assigned_cluster,total_aliases,total_occurrences
149,6_74_2_1_1,17,115
116,6_19_1,15,605
108,6_29,14,1076
106,772_1_1,14,106
213,673_1,14,49
222,6_2_4_2,12,122
51,69_1,12,165
63,1636_1,9,23
83,26_1,8,70
6,235_2,8,63
